In [3]:
import json
import re
from pathlib import Path

# ==============================
# 🔧 可配置参数
# ==============================
input_path = "/projects/b1222/userdata/jianshu/chengxuan/ProgressLM/data/sft_data/sft_all.jsonl"      # 原始文件路径
output_path = "/projects/b1222/userdata/jianshu/chengxuan/ProgressLM/data/sft_data/sft_nothink_all.jsonl"    # 输出文件路径（或文件夹路径）

# ==============================
# 📜 替换模板
# ==============================
VISUAL_TASK = """Your task:
Given a set of demonstration images showing the task progression from start (0%) to completion (100%), and a current state image, analyze them and estimate the current progress. Output only a single number between 0 and 100, representing the estimated progress percentage.
No text, no explanation — output the number only."""

TEXT_TASK = """Your task:
Given a textual demonstration (text_demo) describing how the task progresses from start (0%) to completion (100%), and a current state image, analyze their semantic and visual correspondence to estimate the current progress. Output only a single number between 0 and 100, representing the estimated progress percentage. Do not include any text, reasoning, or labels — output the number only."""

# ==============================
# 🔍 替换逻辑
# ==============================
def replace_user_task(user_content: str, new_block: str) -> str:
    """
    从 'Your task:' 开始删除到末尾并替换为 new_block。
    忽略大小写与多余空格。
    """
    # (?i) 忽略大小写，匹配 Your task: 后任意字符直到末尾
    pattern = re.compile(r"(?is)your\s+task\s*:.*", re.DOTALL)
    new_content, n = pattern.subn(new_block, user_content)
    return new_content, n > 0


def process_jsonl(input_path, output_path):
    total, modified = 0, 0
    results = []

    with open(input_path, "r", encoding="utf-8") as fin:
        for line in fin:
            total += 1
            if not line.strip():
                continue
            try:
                data = json.loads(line)
            except json.JSONDecodeError:
                results.append(line.strip())
                continue

            # 判断 demo 类型
            images = data.get("images", [])
            is_visual = len(images) > 1

            changed = False
            for msg in data.get("messages", []):
                if msg.get("role") == "user" and "task" in msg.get("content", "").lower():
                    new_block = VISUAL_TASK if is_visual else TEXT_TASK
                    new_content, replaced = replace_user_task(msg["content"], new_block)
                    if replaced:
                        msg["content"] = new_content
                        changed = True

            if changed:
                modified += 1

            results.append(json.dumps(data, ensure_ascii=False))

    with open(output_path, "w", encoding="utf-8") as fout:
        fout.write("\n".join(results) + "\n")

    print(f"✅ {Path(input_path).name}: 共 {total} 行，修改 {modified} 行 → 输出至 {output_path}")

# ==============================
# 🚀 运行脚本
# ==============================
process_jsonl(input_path, output_path)


✅ sft_all.jsonl: 共 18580 行，修改 18580 行 → 输出至 /projects/b1222/userdata/jianshu/chengxuan/ProgressLM/data/sft_data/sft_nothink_all.jsonl


In [ ]:
import json
import re
from pathlib import Path

# ==============================
# 🔧 可配置路径
# ==============================
input_path = "/projects/b1222/userdata/jianshu/chengxuan/ProgressLM/data/sft_data/sft_1.jsonl"       # 输入文件路径
output_path = "/projects/b1222/userdata/jianshu/chengxuan/ProgressLM/data/sft_data/sft_nothink.jsonl"  # 输出文件路径

# ==============================
# ⚙️ 提取逻辑
# ==============================
def extract_score(text: str) -> str:
    """
    从 assistant 的 content 中提取 <score>...</score> 里的数字部分。
    若未找到，则返回空字符串。
    """
    match = re.search(r"<score>\s*([\d.]+)\s*%?\s*</score>", text, re.IGNORECASE)
    if match:
        return match.group(1).strip()
    return ""

def process_jsonl(in_path, out_path):
    total, modified = 0, 0
    output_lines = []

    with open(in_path, "r", encoding="utf-8") as f:
        for line in f:
            total += 1
            if not line.strip():
                continue
            try:
                data = json.loads(line)
            except json.JSONDecodeError:
                output_lines.append(line.strip())
                continue

            changed = False
            for msg in data.get("messages", []):
                if msg.get("role") == "assistant":
                    score = extract_score(msg.get("content", ""))
                    if score:
                        msg["content"] = score
                        changed = True

            if changed:
                modified += 1
            output_lines.append(json.dumps(data, ensure_ascii=False))

    with open(out_path, "w", encoding="utf-8") as f:
        f.write("\n".join(output_lines) + "\n")

    print(f"✅ {Path(in_path).name}: 共 {total} 行，提取并修改 {modified} 行 → 输出至 {out_path}")

# ==============================
# 🚀 执行
# ==============================
process_jsonl(input_path, output_path)
